In [3]:
import time
from pyspark.sql.functions import col, avg, max, when
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import col, avg, when

% TODO parametrize your delta table e.g., name_of_your_lakehouse.delta_table_name
% true_big_table is based on yellow_tripdata_2015-01.csv

df = spark.sql("SELECT * FROM Bronze.true_big_table")
df.count()
# 815,935,104 = 815 mln of rows
print("Step 1")
# Define the segments for trip_distance
segments = [0, 5, 10, 15, 20, 25]

# Add a new column for segment based on trip_distance
df_with_segment = df.withColumn("distance_segment", when(
    col("trip_distance").isNull(), None
).otherwise(
    when(col("trip_distance") <= segments[0], "<= " + str(segments[0]))
    .when(col("trip_distance") <= segments[1], str(segments[0]+0.01) + " - " + str(segments[1]))
    .when(col("trip_distance") <= segments[2], str(segments[1]+0.01) + " - " + str(segments[2]))
    .when(col("trip_distance") <= segments[3], str(segments[2]+0.01) + " - " + str(segments[3]))
    .when(col("trip_distance") <= segments[4], str(segments[3]+0.01) + " - " + str(segments[4]))
    .otherwise(">= " + str(segments[4]+0.01))
))
print("Step 2")
# Calculate average fare_amount and tip_amount per segment
avg_amounts_by_segment = df_with_segment.groupBy("distance_segment").agg(
    avg("fare_amount").alias("avg_fare_amount"),
    avg("tip_amount").alias("avg_tip_amount")
)

# Display the result
avg_amounts_by_segment.count()

# Define the segments for trip_distance
segments = [0, 5, 10, 15, 20, 25]
print("Step 3")
# Add a new column for segment based on trip_distance
df_with_segment = df.withColumn("distance_segment", when(
    col("trip_distance").isNull(), None
).otherwise(
    when(col("trip_distance") <= segments[0], "<= " + str(segments[0]))
    .when(col("trip_distance") <= segments[1], str(segments[0]+0.01) + " - " + str(segments[1]))
    .when(col("trip_distance") <= segments[2], str(segments[1]+0.01) + " - " + str(segments[2]))
    .when(col("trip_distance") <= segments[3], str(segments[2]+0.01) + " - " + str(segments[3]))
    .when(col("trip_distance") <= segments[4], str(segments[3]+0.01) + " - " + str(segments[4]))
    .otherwise(">= " + str(segments[4]+0.01))
))

# Calculate average fare_amount, tip_amount, and maximum values per segment
avg_amounts_by_segment = df_with_segment.groupBy("distance_segment").agg(
    avg("fare_amount").alias("avg_fare_amount"),
    avg("tip_amount").alias("avg_tip_amount"),
    max("fare_amount").alias("max_fare_amount"),
    max("tip_amount").alias("max_tip_amount")
)
print("Step 4")
# Order the resulting DataFrame by the first column
ordered_avg_amounts_by_segment = avg_amounts_by_segment.withColumn("distance_segment",
    F.when(col("distance_segment").startswith("<"), F.split(col("distance_segment"), " ")[1])
    .otherwise(F.split(col("distance_segment"), " - ")[0])
).orderBy(col("distance_segment").cast("float"))

# Display the result
ordered_avg_amounts_by_segment.count()

print("Step 5")

# Compute the average trip distance by VendorID
avg_distance = df.groupBy('VendorID').avg('trip_distance')
avg_distance.count()

# Identify the top 10 locations with the highest average tip amount
top_locations = df.groupBy('pickup_longitude', 'pickup_latitude').avg('tip_amount')
top_locations = top_locations.sort('avg(tip_amount)', ascending=False).limit(10)
top_locations.count()

print("Step 6")
# Compute the total revenue by payment type
total_revenue = df.groupBy('payment_type').sum('total_amount')
total_revenue.count()


StatementMeta(, bb9d753a-0434-4290-b0d5-543c1c0f738c, 5, Finished, Cancelled)

0
